In [9]:
from tqdm import tqdm
import numpy as np
import MDAnalysis as mda

import pandas as pd
from pathlib import Path
import MDAnalysis.transformations as trans

import os
import sys
import matplotlib.pyplot as plt
from dotenv import load_dotenv
load_dotenv()


False

In [10]:
# Load simulations
step_size = 500

class Peptide:
    def __init__(self, xtc_file_path, tpr_file_path, peptide_number, amino_acid_count):
        self.xtc_file_path = xtc_file_path
        self.tpr_file_path = tpr_file_path
        self.peptide_number = peptide_number
        self.amino_acid_count = amino_acid_count
        self.u = mda.Universe(tpr_file_path, xtc_file_path)
        self.u.trajectory.add_transformations(trans.unwrap(self.u.select_atoms(f"backbone")))
        self.protein_atoms = self.u.select_atoms("protein")
        self.prot_residues = self.protein_atoms.residues
        self.res_names = self.prot_residues.resnames
        self.res_ids = self.prot_residues.residues.resids
        self.resid_maps = {i+1:j for i, j in enumerate(self.res_ids)}
        self.pep_dict = {
            k: ((k-1)*amino_acid_count+1, k*amino_acid_count) for k in range(1, peptide_number+1)
        }
        self.pep_dict = {k: (self.resid_maps[i[0]], self.resid_maps[i[1]]) for k,i in self.pep_dict.items()}

        self.start_id = self.pep_dict[1][0]
        self.end_id = self.pep_dict[1][1]


    def load_traj(self):
        stop_sim = None
        start, stop_sim, step = self.u.trajectory.check_slice_indices(None, None, None)

        frames = np.arange(start, stop_sim, step_size)
        n_frames = frames.size

        frames_pd = pd.DataFrame(frames)
        # frames_pd.to_csv(f"{base_folder}/frames.csv")
        return frames, n_frames
    


In [11]:
WF2_8_1 = Peptide(
    "/Volumes/miru_back/young_concentration_WF2/WF2_8/md_0_1_combined_first500ns_pbc.xtc",
    "/Volumes/miru_back/young_concentration_WF2/WF2_8/md_0_1.tpr",
    8, 25
)

WF2_16_1 = Peptide(
    "/Volumes/miru_back/young_concentration_WF2/WF2_16/md_0_1_combined_first500ns_pbc.xtc",
    "/Volumes/miru_back/young_concentration_WF2/WF2_16/md_0_1.tpr",
    16, 25
)

WF2_24_1 = Peptide(
    "/Volumes/miru_back/young_concentration_WF2/WF2_24/md_0_1_combined_first500ns_pbc.xtc",
    "/Volumes/miru_back/young_concentration_WF2/WF2_24/md_0_1.tpr",
    24, 25
)
WF2_32_1 = Peptide(
    "/Volumes/miru_back/young_concentration_WF2/WF2_32/md_0_1_combined_first500ns_pbc.xtc",
    "/Volumes/miru_back/young_concentration_WF2/WF2_32/md_0_1.tpr",
    32, 25
)

WF2_8_2 = Peptide(
    "/Volumes/miru_back/young_concentration_WF2/popg_replicas/WF2_8_pg_2/md_0_1_combined_first500ns_pbc.xtc",
     "/Volumes/miru_back/young_concentration_WF2/popg_replicas/WF2_8_pg_2/md_0_1.tpr",
    8, 25
)

WF2_16_2 = Peptide(
    "/Volumes/miru_back/young_concentration_WF2/popg_replicas/WF2_16_pg_2/md_0_1_combined_first500ns_pbc.xtc",
    "/Volumes/miru_back/young_concentration_WF2/popg_replicas/WF2_16_pg_2/md_0_1.tpr",
    16, 25
)

WF2_24_2 = Peptide(
   "/Volumes/miru_back/young_concentration_WF2/popg_replicas/WF2_24_pg_2/md_0_1_combined_first500ns_pbc.xtc",
    "/Volumes/miru_back/young_concentration_WF2/popg_replicas/WF2_24_pg_2/md_0_1.tpr",
    24, 25
)
WF2_32_2 = Peptide(
    "/Volumes/miru_back/young_concentration_WF2/popg_replicas/WF2_32_pg_2/md_0_1_combined_first500ns_pbc.xtc",
    "/Volumes/miru_back/young_concentration_WF2/popg_replicas/WF2_32_pg_2/md_0_1.tpr",
    32, 25
)

In [26]:
def get_membrane_c2(obj):
    # L = leaflet.LeafletFinder(obj.u, 'name C2')

    start_sim, stop_sim, step = obj.u.trajectory.check_slice_indices(None, None, 50)
    frames = np.arange(start_sim, stop_sim, 50)
    n_frames = frames.size
    water_time = []
    for frame_index, _ in tqdm(enumerate(obj.u.trajectory[frames]), total=n_frames):
        p_memb = obj.u.select_atoms(f'resname POPG and name C2').positions[:,[2]].astype(float)
        c2_up = np.min(p_memb[:256])
        c2_low = np.max(p_memb[256:])
        mid_c2 =  (c2_up + c2_low)/2
        water_mols = obj.u.select_atoms(f"(resname SOL and name OW) and (prop abs z < {c2_up}) and (prop abs z > {mid_c2})")
        water_time.append([frame_index, len(water_mols), c2_up, mid_c2])
    return water_time

In [27]:

def calc_and_write_to_file(obj, obj_name):
    results_directory = f"memb_hydr"
    Path(results_directory).mkdir(parents=True, exist_ok=True)
    data = get_membrane_c2(obj)
    df = pd.DataFrame(data, columns=["Time frame", "Waters", "C_up", "c_mid"])
    df.to_csv(f"memb_hydr/memb_hydr_{obj_name}.csv")
    print(f"{obj_name} --- DONE")
    return data

In [29]:
for obj_name, obj in [("WF2_8_1", WF2_8_1), ("WF2_16_1", WF2_16_1), ("WF2_24_1", WF2_24_1), ("WF2_32_1", WF2_32_1),("WF2_8_2", WF2_8_2), ("WF2_16_2", WF2_16_2), ("WF2_24_2", WF2_24_2), ("WF2_32_2", WF2_32_2)]:
      calc_and_write_to_file(obj,obj_name)

100%|██████████| 501/501 [01:46<00:00,  4.68it/s]


WF2_8_1 --- DONE


100%|██████████| 501/501 [01:42<00:00,  4.87it/s]


WF2_16_1 --- DONE


100%|██████████| 500/500 [01:34<00:00,  5.30it/s]


...

WF2_16_2 --- DONE


100%|██████████| 501/501 [01:47<00:00,  4.64it/s]


WF2_24_2 --- DONE


100%|██████████| 501/501 [01:33<00:00,  5.37it/s]

WF2_32_2 --- DONE
